<a href="https://colab.research.google.com/github/pratikesh3232/NLP_practices/blob/main/Mental_health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
priyangshumukherjee_mental_health_text_classification_dataset_path = kagglehub.dataset_download('priyangshumukherjee/mental-health-text-classification-dataset')

print('Data source import complete.')


Using Colab cache for faster access to the 'mental-health-text-classification-dataset' dataset.
Data source import complete.


# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import evaluate


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/mental-health-text-classification-dataset/mental_health_combined_test.csv
/kaggle/input/mental-health-text-classification-dataset/mental_heath_unbanlanced.csv
/kaggle/input/mental-health-text-classification-dataset/mental_heath_feature_engineered.csv


# Load Dataset

In [ ]:
df = pd.read_csv("/kaggle/input/mental-health-text-classification-dataset/mental_heath_unbanlanced.csv")

In [ ]:
df.head()

,Unique_ID,text,status
0,0.0,oh my gosh,Anxiety
1,1.0,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2.0,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3.0,I've shifted my focus to something else but I'...,Anxiety
4,4.0,"I'm restless and restless, it's been a month n...",Anxiety


In [ ]:
df['status'].value_counts()

,count
status,
Normal,18391
Depression,14506
Suicidal,11212
Anxiety,5503


In [ ]:
label_map = {
    "Normal":0,
    "Anxiety": 1,
    "Depression": 2,
    "Suicidal": 3
}

df["label"] = df["status"].map(label_map)


# Load Pretrained BERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


# Tokenization

In [ ]:
def tokenize_f(batch):
    return tokenizer(
        batch['text'],
        truncation=True,
        padding = "max_length",
        max_length=256
    )

In [ ]:
from datasets import Dataset

# Convert pandas DataFrame to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)

# Now, use the map method from the Hugging Face datasets library
tokenized_dataset = hf_dataset.map(
    tokenize_f,
    batched=True,
    remove_columns=df.columns.tolist() # Remove all original DataFrame columns, keeping only tokenized outputs and any added columns like 'label'
)

Map:   0%|          | 0/49612 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.set_format("torch")


# Define Evaluation Metrics

In [ ]:
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "precision": precision.compute(predictions=predictions, references=labels, average="binary")["precision"],
        "recall": recall.compute(predictions=predictions, references=labels, average="binary")["recall"],
        "f1": f1.compute(predictions=predictions, references=labels, average="binary")["f1"],
    }


# Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./bert-sentiment",
    save_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    eval_strategy="steps",
    weight_decay=0.01,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# Trainer

In [ ]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

tokenized_dataset = train_test_split


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics
)

# Train the Model

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


# Final Evaluation

In [ ]:
results = trainer.evaluate()
print(results)
